# Guía Completa de la Librería docxtpl

Esta guía es un recurso exhaustivo para aprender a utilizar la librería `docxtpl` en Python. Esta librería permite generar documentos de Microsoft Word (.docx) dinámicos utilizando plantillas y la sintaxis de Jinja2.

## ¿Qué es docxtpl?

**docxtpl** (Docx Template) es una librería que combina el poder de `python-docx` (para manipular archivos Word) con `jinja2` (un motor de plantillas muy popular en desarrollo web).

**¿Cómo funciona?**
1.  Creas un documento de Word normal (`.docx`) que sirve como **plantilla**.
2.  Dentro del Word, escribes marcadores especiales usando la sintaxis de Jinja2 (ej. `{{ nombre_cliente }}`).
3.  Desde Python, cargas esa plantilla y le pasas un diccionario de datos (contexto).
4.  `docxtpl` reemplaza los marcadores con los datos reales y genera un nuevo documento.

**Características principales:**
*   Sustitución de variables.
*   Bucles (`for`) para tablas y listas dinámicas.
*   Condicionales (`if`) para mostrar/ocultar texto.
*   Inserción de imágenes dinámicas.
*   Texto enriquecido (colores, fuentes) dinámico.
*   Tablas complejas (celdas combinadas).

---
**Requisito:** Necesitas tener instalado Microsoft Word o un visor compatible para abrir los resultados, aunque no es necesario para generarlos con Python.

## 1. Instalación e Importación

Primero, instalamos la librería. `docxtpl` instalará automáticamente sus dependencias (`python-docx` y `jinja2`).

```bash
pip install docxtpl
```

Importaremos `DocxTemplate` para manejar la plantilla y `InlineImage` para insertar imágenes más adelante. También usaremos `Mm` (milímetros) para definir tamaños.

In [ ]:
# Instalación (descomenta si es necesario)
# !pip install docxtpl

from docxtpl import DocxTemplate, InlineImage, RichText
from docx.shared import Mm, Inches, Pt, RGBColor
import os

print("Librerías importadas correctamente.")

## 2. Preparación de la Plantilla

Para que `docxtpl` funcione, **necesitamos un archivo .docx existente**. No podemos crear uno desde cero solo con código (para eso usaríamos `python-docx` puro, pero es más tedioso para diseños complejos).

**Instrucciones:**
1.  Abre Microsoft Word.
2.  Crea un documento nuevo.
3.  Escribe el siguiente texto tal cual (incluyendo las llaves):
    > Hola {{ nombre }},
    >
    > Bienvenido al curso de {{ curso }}.
    >
    > Fecha: {{ fecha }}
4.  Guárdalo como `plantilla_basica.docx` en la misma carpeta que este notebook.

*Nota: Como estamos en un entorno de código, voy a intentar generar una plantilla básica usando `python-docx` para que el ejemplo funcione sin que tengas que salir, pero lo ideal es que tú diseñes tu plantilla en Word.*

In [ ]:
# Generador de plantilla temporal (solo para este ejemplo)
from docx import Document

doc = Document()
doc.add_heading('Plantilla de Prueba', 0)
p = doc.add_paragraph('Hola ')
p.add_run('{{ nombre }}').bold = True
p.add_run(',')

doc.add_paragraph('Bienvenido al curso de {{ curso }}.')
doc.add_paragraph('Fecha: {{ fecha }}')

doc.save('plantilla_basica.docx')
print("Plantilla 'plantilla_basica.docx' creada exitosamente.")

## 3. Renderizado de Variables Simples

Una vez tenemos la plantilla, el proceso es:
1.  Cargar la plantilla con `DocxTemplate`.
2.  Crear un diccionario (`context`) con los datos. Las claves deben coincidir con los nombres dentro de `{{ }}`.
3.  Llamar a `render(context)`.
4.  Guardar el resultado con `save()`.

In [ ]:
# 1. Cargar plantilla
doc = DocxTemplate("plantilla_basica.docx")

# 2. Crear contexto
context = {
    'nombre': 'Daniel Espitia',
    'curso': 'Python Avanzado',
    'fecha': '25 de Noviembre de 2025'
}

# 3. Renderizar
doc.render(context)

# 4. Guardar resultado
doc.save("resultado_basico.docx")
print("Documento 'resultado_basico.docx' generado.")

## 4. Tablas y Bucles (Loops)

Una de las funciones más potentes es llenar tablas dinámicamente.
En la plantilla de Word, usamos la sintaxis de Jinja2 para bucles:

*   Inicio del bucle: `{% for item in lista %}`
*   Fin del bucle: `{% endfor %}`

**Importante:**
*   Si quieres repetir una **fila de una tabla**, debes poner el `{% for ... %}` en la primera celda de la fila y el `{% endfor %}` en la última celda de la misma fila (o usar trucos de comentarios XML, pero lo más fácil es poner los tags dentro de las celdas).
*   O mejor aún, usar la sintaxis de fila completa: `{% tr for item in lista %}` (esto requiere configuración especial) o simplemente poner el tag al principio y final de la fila.

Vamos a crear una plantilla para una tabla de notas.

In [ ]:
# Crear plantilla con tabla (simulada)
doc = Document()
doc.add_heading('Reporte de Notas', 0)
table = doc.add_table(rows=2, cols=3)

# Encabezados
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Estudiante'
hdr_cells[1].text = 'Materia'
hdr_cells[2].text = 'Nota'

# Fila de plantilla (aquí es donde ocurre la magia en Word real)
# En Word escribirías:
# Celda 1: {% for a in alumnos %}{{ a.nombre }}
# Celda 2: {{ a.materia }}
# Celda 3: {{ a.nota }}{% endfor %}

row_cells = table.rows[1].cells
row_cells[0].text = '{% for a in alumnos %}{{ a.nombre }}'
row_cells[1].text = '{{ a.materia }}'
row_cells[2].text = '{{ a.nota }}{% endfor %}'

doc.save('plantilla_tabla.docx')

# Renderizar la tabla
doc_tpl = DocxTemplate("plantilla_tabla.docx")

lista_alumnos = [
    {'nombre': 'Juan', 'materia': 'Matemáticas', 'nota': 9.5},
    {'nombre': 'Maria', 'materia': 'Física', 'nota': 8.0},
    {'nombre': 'Pedro', 'materia': 'Química', 'nota': 6.5},
]

context = {'alumnos': lista_alumnos}

doc_tpl.render(context)
doc_tpl.save("resultado_tabla.docx")
print("Documento 'resultado_tabla.docx' generado con filas dinámicas.")

## 5. Condicionales (If/Else)

Puedes mostrar u ocultar partes del documento según una condición.

Sintaxis:
```jinja2
{% if aprobado %}
    Felicidades, has aprobado el curso.
{% else %}
    Lo sentimos, debes repetir el examen.
{% endif %}
```

In [ ]:
# Crear plantilla con condicional
doc = Document()
doc.add_paragraph('Resultado del examen:')
doc.add_paragraph('{% if nota >= 60 %}APROBADO{% else %}REPROBADO{% endif %}')
doc.save('plantilla_condicional.docx')

# Renderizar caso Aprobado
doc_tpl = DocxTemplate("plantilla_condicional.docx")
doc_tpl.render({'nota': 85})
doc_tpl.save("resultado_aprobado.docx")

# Renderizar caso Reprobado
doc_tpl = DocxTemplate("plantilla_condicional.docx")
doc_tpl.render({'nota': 45})
doc_tpl.save("resultado_reprobado.docx")

print("Documentos condicionales generados.")

## 6. Imágenes Dinámicas

Para insertar imágenes que cambian según el contexto, usamos `InlineImage`.
En la plantilla de Word, solo necesitas poner un marcador normal: `{{ mi_imagen }}`.
En Python, debes pasar un objeto `InlineImage` en lugar de un string.

In [ ]:
# Crear plantilla para imagen
doc = Document()
doc.add_paragraph('Aquí está tu imagen dinámica:')
doc.add_paragraph('{{ imagen_dinamica }}')
doc.save('plantilla_imagen.docx')

# Necesitamos una imagen real para probar.
# Si no tienes una, este código fallará. Asegúrate de tener 'logo.png' o cambia el nombre.
# Voy a crear una imagen dummy pequeña con PIL si no existe
try:
    from PIL import Image
    if not os.path.exists('test_image.png'):
        img = Image.new('RGB', (100, 100), color = 'red')
        img.save('test_image.png')
    
    doc_tpl = DocxTemplate("plantilla_imagen.docx")
    
    # Crear objeto InlineImage
    # width=Mm(20) define el ancho en milímetros
    imagen = InlineImage(doc_tpl, 'test_image.png', width=Mm(50))
    
    context = {'imagen_dinamica': imagen}
    
    doc_tpl.render(context)
    doc_tpl.save("resultado_imagen.docx")
    print("Documento con imagen generado.")

except ImportError:
    print("Se requiere la librería Pillow (PIL) para generar la imagen de prueba.")
except Exception as e:
    print(f"Error: {e}")

## 7. Texto Enriquecido (RichText)

A veces quieres cambiar el color o el estilo de una variable dinámicamente (por ejemplo, poner un número en rojo si es negativo).
Para esto usamos `RichText`.

En la plantilla: `{{ variable_con_formato }}`
En Python: pasamos un objeto `RichText`.

In [ ]:
# Crear plantilla
doc = Document()
doc.add_paragraph('El estado del sistema es: {{ estado }}')
doc.save('plantilla_richtext.docx')

doc_tpl = DocxTemplate("plantilla_richtext.docx")

# Crear RichText
rt = RichText()
rt.add('CRÍTICO', color='#FF0000', bold=True, size=20) # Rojo, negrita, grande
rt.add(' (Revisar inmediatamente)', italic=True, color='#0000FF') # Azul, cursiva

context = {'estado': rt}

doc_tpl.render(context)
doc_tpl.save("resultado_richtext.docx")
print("Documento con RichText generado.")

## 8. Sub-documentos

Puedes incluir un documento Word completo dentro de otro. Esto es útil si tienes partes reutilizables (como términos y condiciones) o si quieres combinar varios reportes.

En la plantilla: `{{ subdocumento }}`
En Python: `doc.new_subdoc('ruta.docx')`

In [ ]:
# Crear documento principal
doc = Document()
doc.add_paragraph('Inicio del reporte principal.')
doc.add_paragraph('{{ contenido_extra }}')
doc.add_paragraph('Fin del reporte.')
doc.save('plantilla_master.docx')

# Crear sub-documento
sub = Document()
sub.add_heading('Anexo A', level=2)
sub.add_paragraph('Este contenido viene de otro archivo Word.')
sub.save('anexo.docx')

# Renderizar
doc_tpl = DocxTemplate("plantilla_master.docx")

# Cargar el subdocumento
sub_doc = doc_tpl.new_subdoc("anexo.docx")

context = {'contenido_extra': sub_doc}

doc_tpl.render(context)
doc_tpl.save("resultado_subdoc.docx")
print("Documento fusionado generado.")

## 9. Filtros Jinja2 Personalizados

Jinja2 permite modificar variables antes de mostrarlas usando filtros (ej. `{{ precio | currency }}`). Podemos definir nuestros propios filtros en Python.

In [ ]:
import jinja2

# Función de filtro
def formato_moneda(valor):
    return f"${valor:,.2f}"

# Crear plantilla
doc = Document()
doc.add_paragraph('El precio total es: {{ precio | moneda }}')
doc.save('plantilla_filtro.docx')

doc_tpl = DocxTemplate("plantilla_filtro.docx")

# Registrar el filtro en el entorno Jinja2 de docxtpl
jinja_env = jinja2.Environment()
jinja_env.filters['moneda'] = formato_moneda

# Renderizar pasando el entorno personalizado
doc_tpl.render({'precio': 12345.6789}, jinja_env)
doc_tpl.save("resultado_filtro.docx")
print("Documento con filtro personalizado generado.")